In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/new-tokenizer/merges.txt
/kaggle/input/new-tokenizer/tokenizer.json
/kaggle/input/new-tokenizer/vocab.json
/kaggle/input/new-tokenizer/tokenizer_config.json
/kaggle/input/new-tokenizer/special_tokens_map.json
/kaggle/input/new-tokenizer/added_tokens.json
/kaggle/input/neurips-open-polymer-prediction-2025/sample_submission.csv
/kaggle/input/neurips-open-polymer-prediction-2025/train.csv
/kaggle/input/neurips-open-polymer-prediction-2025/test.csv
/kaggle/input/neurips-open-polymer-prediction-2025/train_supplement/dataset2.csv
/kaggle/input/neurips-open-polymer-prediction-2025/train_supplement/dataset4.csv
/kaggle/input/neurips-open-polymer-prediction-2025/train_supplement/dataset1.csv
/kaggle/input/neurips-open-polymer-prediction-2025/train_supplement/dataset3.csv


In [2]:
import pandas as pd
train_csv = pd.read_csv("/kaggle/input/neurips-open-polymer-prediction-2025/train.csv")
train_csv['SMILES'][1]

'*Nc1ccc([C@H](CCC)c2ccc(C3(c4ccc([C@@H](CCC)c5ccc(N*)cc5)cc4)CCC(CCCCC)CC3)cc2)cc1'

In [3]:
test_csv = pd.read_csv("/kaggle/input/neurips-open-polymer-prediction-2025/test.csv")
test_csv

,id,SMILES
0,1109053969,*Oc1ccc(C=NN=Cc2ccc(Oc3ccc(C(c4ccc(*)cc4)(C(F)...
1,1422188626,*Oc1ccc(C(C)(C)c2ccc(Oc3ccc(C(=O)c4cccc(C(=O)c...
2,2032016830,*c1cccc(OCCCCCCCCOc2cccc(N3C(=O)c4ccc(-c5cccc6...


In [4]:
Tc_extra = pd.read_csv("/kaggle/input/neurips-open-polymer-prediction-2025/train_supplement/dataset1.csv")
test2 = pd.read_csv("/kaggle/input/neurips-open-polymer-prediction-2025/train_supplement/dataset2.csv")
Tg_extra = pd.read_csv("/kaggle/input/neurips-open-polymer-prediction-2025/train_supplement/dataset3.csv")
FFV_extra = pd.read_csv("/kaggle/input/neurips-open-polymer-prediction-2025/train_supplement/dataset4.csv")

Tc_extra.rename(columns={"TC_mean": "Tc"}, inplace=True)
Tc_extra

,SMILES,Tc
0,*/C(=C(\c1ccccc1)c1ccc(*)cc1)c1ccccc1,0.3380
1,*/C(F)=C(\F)C(F)(C(*)(F)F)C(F)(F)F,0.1020
2,*/C=C(/*)C#CCCCCCCCCCCCCCCCCCCCCC(=O)O,0.4105
3,*/C=C(/*)CCCCCCCCCCCCCCCCCCCCC(=O)O,0.4030
4,*/C=C/*,0.5260
...,...,...
869,CCNC(=O)OCCCCC(*)=C=C=C(*)CCCCOC(=O)NCC,0.2600
870,Cc1ccc(cc1)S(=O)(=O)OCCCCC(*)=C=C=C(*)CCCCOS(=...,0.2100
871,FC(F)(F)C(C1=CC2=C(OC(=N2)C2=CC=C(OC3=C4C5C6=C...,0.1090
872,FC(F)(F)C(C1=CC2=C(OC(=N2)C2=CC=C(OC3=C4C5C6=C...,0.1760


In [5]:
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained("/kaggle/input/new-tokenizer")


In [6]:

input_text = 'C1=CC2=C(C(=C1)[O-])NC(=CC2=O)C(=O)O'
input_ids = tokenizer(input_text, return_tensors="pt").input_ids


In [7]:
input_text = train_csv['SMILES'][1]
input_ids = tokenizer(input_text, return_tensors="pt").input_ids
input_ids
#outputs = model.generate(input_ids, num_beams=5, max_length=512)
#print(tokenizer.decode(outputs[0], skip_special_tokens=True))

tensor([[ 12, 256,  23,  15,  20,  15,  15,  15,  17,  16,  17,  16,  16,  16,
          18,  15,  21,  15,  15,  15,  17,  16,  26,  17,  15,  32,  15,  15,
          15,  17,  16,  17,  16,  16,  16,  18,  15,  43,  15,  15,  15,  17,
          23, 256,  18,  15,  15,  43,  18,  15,  15,  32,  18,  16,  16,  16,
          17,  16,  16,  16,  16,  16,  18,  16,  16,  26,  18,  15,  15,  21,
          18,  15,  15,  20,  13]])

In [8]:
input_ids.shape

torch.Size([1, 75])

In [9]:
non_null_count = train_csv['FFV'].notnull().sum()
print(non_null_count) 

7030


In [10]:
features = ['SMILES','FFV']
FFV_train_ = train_csv[train_csv['FFV'].notnull()][features].copy()

features = ['SMILES','Tg']
Tg_train_ = train_csv[train_csv['Tg'].notnull()][features].copy()

features = ['SMILES','Tc']
Tc_train_ = train_csv[train_csv['Tc'].notnull()][features].copy()

features = ['SMILES','Density']
Density_train = train_csv[train_csv['Density'].notnull()][features].copy()

features = ['SMILES','Rg']
Rg_train = train_csv[train_csv['Rg'].notnull()][features].copy()


In [11]:
Tc_train_

,SMILES,Tc
0,*CC(*)c1ccccc1C(=O)OCCCCCC,0.205667
10,*c1ccc(-c2ccc3c(c2)C(CCCCCCC#N)(CCCCCCC#N)c2cc...,0.487000
11,*CC(*)c1ccc(C(=O)O)c(C(=O)O)c1,0.171000
31,*CCCCCNC(=O)CCCCC(=O)N*,0.327000
34,*CCCCCCCCCCCCCCCCCCNC(=O)NCCCCCCNC(=O)N*,0.383000
...,...,...
7899,*CCN(*)C(=O)CCCCCCCCCCCCCCC,0.336000
7905,*CC(*)c1ccccc1F,0.174667
7943,*CC(*)C1CC=CCC1,0.180750
7944,*CC(*)Cl,0.139000


In [12]:
FFV_train = pd.concat([FFV_train_, FFV_extra], ignore_index = True)
Tg_train = pd.concat([Tg_train_, Tg_extra], ignore_index = True)
Tc_train = pd.concat([Tc_train_, Tc_extra], ignore_index = True)
print(FFV_train.shape)
print(Tg_train.shape)
print(Tc_train.shape)
print(Density_train.shape)
print(Rg_train.shape)

(7892, 2)
(557, 2)
(1611, 2)
(613, 2)
(614, 2)


In [13]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

2025-08-17 19:11:17.529459: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755457877.731673      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755457877.793051      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [14]:
model_FFV = Sequential([
    Dense(150, activation='linear', input_shape=(75,)),
    Dropout(0.3),
    Dense(150, activation='linear'),
    Dropout(0.3),                                                                           
    Dense(1, activation='linear')
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
I0000 00:00:1755457888.688111      19 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


In [15]:
X = tokenizer(
    FFV_train["SMILES"].tolist(),
    padding="max_length",
    truncation=True,
    max_length=75,  
    return_tensors="np" 
).input_ids

print(X.shape)

(7892, 75)


In [16]:
from sklearn.model_selection import train_test_split
Y = FFV_train['FFV'].values
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42, shuffle=True)

In [17]:
# model_FFV.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
#              loss = "mae",                  
#     metrics=['mae','mse','mape'])

In [18]:
import tensorflow as tf

model_FFV.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss="mae",                 # MAE
    metrics=["mae"]             # track MAE
)
sample_weights = np.ones(len(X_train))
# use sample_weight to turn MAE into WMAE over the batch
history = model_FFV.fit(
    X_train, Y_train,
    sample_weight=sample_weights,   
    validation_data=(X_test, Y_test),
    epochs=100,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor="val_mae", mode="min", patience=10, restore_best_weights=True
        ),
        tf.keras.callbacks.ReduceLROnPlateau(
            monitor="val_mae", mode="min", factor=0.5, patience=3
        )
    ],
    batch_size=32
)


Epoch 1/100


I0000 00:00:1755457892.425566      64 service.cc:148] XLA service 0x7d154400ce40 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1755457892.426001      64 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1755457892.640653      64 cuda_dnn.cc:529] Loaded cuDNN version 90300


 68/198 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 37.7033 - mae: 37.7033

I0000 00:00:1755457894.004863      64 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


198/198 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step - loss: 29.2922 - mae: 29.2922 - val_loss: 4.6818 - val_mae: 4.6818 - learning_rate: 0.0010
Epoch 2/100
198/198 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 11.2456 - mae: 11.2456 - val_loss: 2.0230 - val_mae: 2.0230 - learning_rate: 0.0010
Epoch 3/100
198/198 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 5.0422 - mae: 5.0422 - val_loss: 0.6085 - val_mae: 0.6085 - learning_rate: 0.0010
Epoch 4/100
198/198 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 1.2692 - mae: 1.2692 - val_loss: 0.1750 - val_mae: 0.1750 - learning_rate: 0.0010
Epoch 5/100
198/198 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.3046 - mae: 0.3046 - val_loss: 0.1655 - val_mae: 0.1655 - learning_rate: 0.0010
Epoch 6/100
198/198 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.2762 - mae: 0.2762 - val_loss: 0.1907 - val_mae: 0.1907 - learning_rate: 0.0010
Epoch 7/100
198/198 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.2906 - mae: 0.2906 - val_loss: 0.1787 - val_mae: 0.1787 - learning_rate: 0.0010
Epoch 

In [19]:
#model_FFV.fit(X_train, Y_train, epochs = 50, batch_size=32)

In [20]:
results = model_FFV.evaluate(X_test, Y_test)
print(f"Test loss: {results[0]}")
print(f"Test MAE: {results[1]}")
#print(f"Test MSE: {results[2]}")
##print(f"Test MAPE: {results[3]}")

50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0201 - mae: 0.0201
Test loss: 0.019469911232590675
Test MAE: 0.019469911232590675


In [21]:

X_sub = tokenizer(
    test_csv["SMILES"].tolist(),
    padding="max_length",
    truncation=True,
    max_length=75,  
    return_tensors="np"  
).input_ids

Y_FFV_sub = model_FFV.predict(X_sub)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 187ms/step


In [22]:
Y_FFV_sub

array([[0.3679846],
       [0.3663791],
       [0.3698545]], dtype=float32)

In [23]:
model_Tg = Sequential([
    Dense(64, activation='relu', input_shape=(75,), kernel_regularizer=tf.keras.regularizers.l2(1e-4)),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dense(1, activation='linear')
])

X_Tg = tokenizer(
    Tg_train["SMILES"].tolist(),
    padding="max_length",
    truncation=True,
    max_length=75,  
    return_tensors="np"  
).input_ids

print(X_Tg.shape)

model_Tg.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
             loss = "mae",                  
    metrics=['mae','mse','mape'])

(557, 75)


In [24]:
model_Tc = Sequential([
    Dense(150, activation='relu', input_shape=(75,)),
    Dropout(0.3),
    Dense(75, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='linear')
])

X_Tc = tokenizer(
    Tc_train["SMILES"].tolist(),
    padding="max_length",
    truncation=True,
    max_length=75,  
    return_tensors="np"  
).input_ids

print(X_Tc.shape)

model_Tc.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
             loss = "mae",                  
    metrics=['mae','mse','mape'])

(1611, 75)


In [25]:
model_Density = Sequential([
    Dense(75, activation='relu', input_shape=(75,)),
    Dropout(0.3),
    Dense(50, activation = 'relu'),
    Dropout(0.3),
    Dense(1, activation='linear')
])

X_Density = tokenizer(
    Density_train["SMILES"].tolist(),
    padding="max_length",
    truncation=True,
    max_length=75,  
    return_tensors="np"  
).input_ids

print(X_Density.shape)


model_Density.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
             loss = "mae",                  
    metrics=['mae','mse','mape'])

(613, 75)


In [26]:
model_Rg = Sequential([
    Dense(75, activation='linear', input_shape=(75,)),
    Dropout(0.3),
    Dense(50, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='linear')
])

X_Rg = tokenizer(
    Rg_train["SMILES"].tolist(),
    padding="max_length",
    truncation=True,
    max_length=75,  
    return_tensors="np"  
).input_ids

print(X_Rg.shape)


model_Rg.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
             loss = "mae",                  
    metrics=['mae','mse','mape'])

(614, 75)


In [27]:
Y_Tg = Tg_train['Tg'].values
X_train_Tg, X_test_Tg, Y_train_Tg, Y_test_Tg = train_test_split(X_Tg, Y_Tg, test_size=0.2, random_state=42)
Y_Tc = Tc_train['Tc'].values
X_train_Tc, X_test_Tc, Y_train_Tc, Y_test_Tc = train_test_split(X_Tc, Y_Tc, test_size=0.2, random_state=42)
Y_Density = Density_train['Density'].values
X_train_D, X_test_D, Y_train_D, Y_test_D = train_test_split(X_Density, Y_Density, test_size=0.2, random_state=42)
Y_Rg = Rg_train['Rg'].values
X_train_Rg, X_test_Rg, Y_train_Rg, Y_test_Rg = train_test_split(X_Rg, Y_Rg, test_size=0.2, random_state=42)

In [28]:
model_Tg.fit(X_train_Tg, Y_train_Tg, epochs=30, batch_size=32)

Epoch 1/30
14/14 ━━━━━━━━━━━━━━━━━━━━ 4s 79ms/step - loss: 101.9384 - mae: 101.9313 - mape: 272.8516 - mse: 17492.8105
Epoch 2/30
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 79.3122 - mae: 79.3051 - mape: 419.8241 - mse: 10145.4346
Epoch 3/30
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 81.2199 - mae: 81.2129 - mape: 422.5101 - mse: 10275.7793
Epoch 4/30
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 77.1649 - mae: 77.1579 - mape: 312.5265 - mse: 9872.3135 
Epoch 5/30
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 74.8692 - mae: 74.8622 - mape: 266.5199 - mse: 9442.5645 
Epoch 6/30
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 82.2255 - mae: 82.2185 - mape: 463.4227 - mse: 11453.6045  
Epoch 7/30
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 76.2217 - mae: 76.2147 - mape: 381.4507 - mse: 9252.7803 
Epoch 8/30
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 76.3006 - mae: 76.2936 - mape: 317.6399 - mse: 9595.6895  
Epoch 9/30
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 76.6991 

In [29]:
history = model_Tg.fit(
    X_train_Tg, Y_train_Tg,
    validation_data=(X_test_Tg, Y_test_Tg),
    epochs=200,
    batch_size=16,   # smaller batch since small dataset
    callbacks=[
        tf.keras.callbacks.EarlyStopping(monitor="val_mae", patience=20, restore_best_weights=True),
        tf.keras.callbacks.ReduceLROnPlateau(monitor="val_mae", factor=0.5, patience=5)
    ]
)

Epoch 1/200
28/28 ━━━━━━━━━━━━━━━━━━━━ 3s 58ms/step - loss: 64.2796 - mae: 64.2721 - mape: 557.7597 - mse: 6949.1924 - val_loss: 79.6309 - val_mae: 79.6233 - val_mape: 1112.9401 - val_mse: 10898.7520 - learning_rate: 0.0010
Epoch 2/200
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 66.2707 - mae: 66.2632 - mape: 186.6380 - mse: 7698.2930 - val_loss: 82.3661 - val_mae: 82.3585 - val_mape: 1257.4412 - val_mse: 11181.4043 - learning_rate: 0.0010
Epoch 3/200
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 65.4223 - mae: 65.4148 - mape: 326.4564 - mse: 7233.3789 - val_loss: 81.9305 - val_mae: 81.9229 - val_mape: 1238.3351 - val_mse: 11036.7969 - learning_rate: 0.0010
Epoch 4/200
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 62.3911 - mae: 62.3835 - mape: 491.4695 - mse: 6719.1001 - val_loss: 80.0078 - val_mae: 80.0002 - val_mape: 1156.3593 - val_mse: 11060.8809 - learning_rate: 0.0010
Epoch 5/200
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 67.3297 - mae: 67.3220 - mape: 359.4252 - mse: 7

In [30]:
results = model_Tg.evaluate(X_test_Tg, Y_test_Tg)
print(f"Test loss: {results[0]}")
print(f"Test MAE: {results[1]}")
print(f"Test MSE: {results[2]}")
print(f"Test MAPE: {results[3]}")

4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 58ms/step - loss: 78.0221 - mae: 78.0146 - mape: 868.6573 - mse: 10048.6885
Test loss: 79.630859375
Test MAE: 79.62334442138672
Test MSE: 10898.7509765625
Test MAPE: 1112.93994140625


In [31]:
# model_Tc.fit(X_train_Tc, Y_train_Tc, epochs=50, batch_size=32)

In [32]:
history = model_Tc.fit(
    X_train_Tc, Y_train_Tc,
    validation_data=(X_test_Tc, Y_test_Tc),
    epochs=200,
    batch_size=32,   
    callbacks=[
        tf.keras.callbacks.EarlyStopping(monitor="val_mae", patience=20, restore_best_weights=True),
        tf.keras.callbacks.ReduceLROnPlateau(monitor="val_mae", factor=0.5, patience=5)
    ]
)

Epoch 1/200
41/41 ━━━━━━━━━━━━━━━━━━━━ 5s 50ms/step - loss: 22.9485 - mae: 22.9485 - mape: 10373.6895 - mse: 900.3760 - val_loss: 4.1299 - val_mae: 4.1299 - val_mape: 1857.2899 - val_mse: 21.0865 - learning_rate: 0.0010
Epoch 2/200
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 7.8331 - mae: 7.8331 - mape: 3507.5408 - mse: 104.6830 - val_loss: 1.2573 - val_mae: 1.2573 - val_mape: 499.6388 - val_mse: 3.1083 - learning_rate: 0.0010
Epoch 3/200
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 3.3090 - mae: 3.3090 - mape: 1427.6818 - mse: 21.5119 - val_loss: 0.3502 - val_mae: 0.3502 - val_mape: 140.9015 - val_mse: 0.2272 - learning_rate: 0.0010
Epoch 4/200
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 1.2812 - mae: 1.2812 - mape: 523.9532 - mse: 3.9121 - val_loss: 0.1904 - val_mae: 0.1904 - val_mape: 73.1531 - val_mse: 0.0697 - learning_rate: 0.0010
Epoch 5/200
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.5642 - mae: 0.5642 - mape: 224.2478 - mse: 1.1389 - val_loss: 0.1300 - val_mae: 0.

In [33]:
results = model_Tc.evaluate(X_test_Tc, Y_test_Tc)
print(f"Test loss: {results[0]}")
print(f"Test MAE: {results[1]}")
print(f"Test MSE: {results[2]}")
print(f"Test MAPE: {results[3]}")

11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0559 - mae: 0.0559 - mape: 24.5599 - mse: 0.0055 
Test loss: 0.05601029098033905
Test MAE: 0.05601029098033905
Test MSE: 0.0056556216441094875
Test MAPE: 24.346595764160156


In [34]:
# model_Density.fit(X_train_D, Y_train_D, epochs = 25, batch_size=16)

In [35]:
history = model_Density.fit(
    X_train_D, Y_train_D,
    validation_data=(X_test_D, Y_test_D),
    epochs=200,
    batch_size=16,   
    callbacks=[
        tf.keras.callbacks.EarlyStopping(monitor="val_mae", patience=20, restore_best_weights=True),
        tf.keras.callbacks.ReduceLROnPlateau(monitor="val_mae", factor=0.5, patience=5)
    ]
)

Epoch 1/200
31/31 ━━━━━━━━━━━━━━━━━━━━ 5s 67ms/step - loss: 25.9818 - mae: 25.9818 - mape: 2678.5098 - mse: 1078.6342 - val_loss: 4.8073 - val_mae: 4.8073 - val_mape: 483.4142 - val_mse: 50.9651 - learning_rate: 0.0010
Epoch 2/200
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 12.4504 - mae: 12.4504 - mape: 1290.0627 - mse: 251.4652 - val_loss: 4.2973 - val_mae: 4.2973 - val_mape: 440.2481 - val_mse: 28.6329 - learning_rate: 0.0010
Epoch 3/200
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 8.0185 - mae: 8.0185 - mape: 841.2750 - mse: 108.8547 - val_loss: 2.1918 - val_mae: 2.1918 - val_mape: 229.3423 - val_mse: 10.8176 - learning_rate: 0.0010
Epoch 4/200
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 5.7846 - mae: 5.7846 - mape: 596.1712 - mse: 64.9717 - val_loss: 1.6098 - val_mae: 1.6098 - val_mape: 167.0380 - val_mse: 5.9855 - learning_rate: 0.0010
Epoch 5/200
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 4.5026 - mae: 4.5026 - mape: 471.7636 - mse: 42.6346 - val_loss: 1.2770 - val_m

In [36]:
results = model_Density.evaluate(X_test_D, Y_test_D)
print(f"Test loss: {results[0]}")
print(f"Test MAE: {results[1]}")
print(f"Test MSE: {results[2]}")
print(f"Test MAPE: {results[3]}")

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step - loss: 0.1081 - mae: 0.1081 - mape: 10.5838 - mse: 0.0198
Test loss: 0.10766863077878952
Test MAE: 0.10766863077878952
Test MSE: 0.020917141810059547
Test MAPE: 10.415931701660156


In [37]:
# model_Rg.fit(X_train_Rg, Y_train_Rg, epochs = 25, batch_size=16)

In [38]:
history = model_Rg.fit(
    X_train_Rg, Y_train_Rg,
    validation_data=(X_test_Rg, Y_test_Rg),
    epochs=200,
    batch_size=16,   
    callbacks=[
        tf.keras.callbacks.EarlyStopping(monitor="val_mae", patience=20, restore_best_weights=True),
        tf.keras.callbacks.ReduceLROnPlateau(monitor="val_mae", factor=0.5, patience=5)
    ]
)

Epoch 1/200
31/31 ━━━━━━━━━━━━━━━━━━━━ 5s 64ms/step - loss: 39.7161 - mae: 39.7161 - mape: 266.3666 - mse: 2527.4841 - val_loss: 10.7530 - val_mae: 10.7530 - val_mape: 66.5835 - val_mse: 182.9625 - learning_rate: 0.0010
Epoch 2/200
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 24.4255 - mae: 24.4255 - mape: 158.7139 - mse: 946.7323 - val_loss: 8.0812 - val_mae: 8.0812 - val_mape: 47.2927 - val_mse: 107.2888 - learning_rate: 0.0010
Epoch 3/200
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 15.3051 - mae: 15.3051 - mape: 97.5432 - mse: 386.6732 - val_loss: 7.1572 - val_mae: 7.1572 - val_mape: 42.9836 - val_mse: 72.0040 - learning_rate: 0.0010
Epoch 4/200
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 11.8473 - mae: 11.8473 - mape: 71.4805 - mse: 249.9698 - val_loss: 5.7359 - val_mae: 5.7359 - val_mape: 35.1431 - val_mse: 47.7297 - learning_rate: 0.0010
Epoch 5/200
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 10.5255 - mae: 10.5255 - mape: 62.9970 - mse: 196.7015 - val_loss: 6.6053 - v

In [39]:
results = model_Rg.evaluate(X_test_Rg, Y_test_Rg)
print(f"Test loss: {results[0]}")
print(f"Test MAE: {results[1]}")
print(f"Test MSE: {results[2]}")
print(f"Test MAPE: {results[3]}")

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step - loss: 3.0529 - mae: 3.0529 - mape: 16.1042 - mse: 18.3067
Test loss: 2.922363519668579
Test MAE: 2.922363519668579
Test MSE: 18.32884407043457
Test MAPE: 15.25938892364502


In [40]:
Y_Tg_sub = model_Tg.predict(X_sub)
Y_Tg_sub

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 189ms/step


array([[135.45674],
       [206.54695],
       [153.89232]], dtype=float32)

In [41]:
Y_Tc_sub = model_Tc.predict(X_sub)
Y_Tc_sub

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 171ms/step


array([[0.24742995],
       [0.2550468 ],
       [0.2333528 ]], dtype=float32)

In [42]:
Y_Density_sub = model_Density.predict(X_sub)
Y_Density_sub

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 195ms/step


array([[0.8925009 ],
       [0.98353463],
       [1.0418092 ]], dtype=float32)

In [43]:
Y_Rg_sub = model_Rg.predict(X_sub)
Y_Rg_sub

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 162ms/step


array([[17.040133],
       [19.170553],
       [16.61875 ]], dtype=float32)

In [44]:
submission = pd.DataFrame({
    "id": test_csv["id"],
    "Tg" : Y_Tg_sub.ravel(),
    "FFV" : Y_FFV_sub.ravel(),
    "Tc" : Y_Tc_sub.ravel(),
    "Density": Y_Density_sub.ravel(),
    "Rg" : Y_Rg_sub.ravel()
})
submission


,id,Tg,FFV,Tc,Density,Rg
0,1109053969,135.456741,0.367985,0.247430,0.892501,17.040133
1,1422188626,206.546951,0.366379,0.255047,0.983535,19.170553
2,2032016830,153.892319,0.369855,0.233353,1.041809,16.618750


In [45]:
submission.to_csv("submission.csv", index=False)
print("Done")

Done
